In [1]:
import numpy as np
import pandas as pd

from patsy import dmatrices, dmatrix
import re
import pickle
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.datasets import make_classification
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC, SVC
from xgboost import XGBClassifier
from collections import Counter
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model, svm
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier

import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.plotting import plot_decision_regions
%matplotlib inline

# make prettier plots
%config InlineBackend.figure_format = 'svg' 

seed = 5
np.random.seed(seed)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
file = open('cleaned_cc_default_data', 'rb')
model_data = pickle.load(file)

### Splitting data into train/test & scaling

In [ ]:
# Use Patsy to create my X Matrix

## QUESTION: need to change variable names to avoid mix up with non clustered variables above?

x_cols = ['my_clusters',
        'age', 'sex', 'marital_status', 'education_level',  
        'bill_amt_1', 'bill_amt_2', 'bill_amt_3', 'bill_amt_4', 'bill_amt_5', 'bill_amt_6', 
        'pay_0', 'pay_2', 'pay_3', 'pay_4', 'pay_5', 'pay_6', 
        'pay_amt_1', 'pay_amt_2', 'pay_amt_3', 'pay_amt_4', 'pay_amt_5', 'pay_amt_6', 
         'limit_balance']
# Add interactions
x_str = x_cols[0]
for i in x_cols[1:]:
    x_str = x_str + ' + ' + i
x_str = x_str + '' # if I want to create new variables, add in empty string

x_patsy = dmatrix(x_str, model_data)
x_patsy.shape

In [ ]:
# Stratified train test split

# CHANGE VARIABLE NAMES?

x_raw_clusters = x_patsy
y_raw_clusters = np.array(model_data['default_payment_next_month'])

sss1 = StratifiedShuffleSplit(n_splits=2, test_size=0.2, random_state=0)
# sss2 = StratifiedShuffleSplit(n_splits=2, test_size=0.25, random_state=0)

sss1.get_n_splits()
for train_index, test_index in sss1.split(x_raw, y_raw):
    x_train_clusters, x_test_clusters = x_raw_clusters[train_index,:], x_raw_clusters[test_index,:]
    y_train_clusters, y_test_clusters = y_raw_clusters[train_index], y_raw_clusters[test_index]

# use this later if I want to get fancy...
# sss2.get_n_splits()
# for train_index, test_index in sss2.split(x_mid, y_mid):
#     x_train, x_val = x_mid[train_index,:], x_mid[test_index,:]
#     y_train, y_val = y_mid[train_index], y_mid[test_index]

In [20]:
# Normalize my variables
scaler = StandardScaler()
scaler.fit(x_train)
x_norm_train = scaler.transform(x_train)
# x_norm_val = scaler.transform(x_val)
x_norm_test = scaler.transform(x_test)

### Final Model

### EDA on Final Model

In [ ]:
# Note: once I've picked my model, start doing EDA to understand how the top features are contributing to the model (e.g., do married people tend to default less often on their pyaments?)
# do this for the top 10 features for Random Forest or Gradient Boosting Trees (or XGBoost)